# Getting Started Code for [Chunin Exams Food Track- CV'2021](https://www.aicrowd.com/challenges/chunin-exams-food-track-cv-2021) on AIcrowd
#### Author : Pulkit Gera

## Download Necessary Packages 📚

In [ ]:
!pip3 install aicrowd-cli

     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 204kB 12.0MB/s 
     |████████████████████████████████| 163kB 31.2MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


## Download Data
The first step is to download out train test data. We will be training a model on the train data and make predictions on test data. We submit our predictions.


In [6]:
API_KEY = "fd1aed7204268f0bae2780fb808a4063" #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

/bin/bash: aicrowd: command not found


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# !aicrowd dataset download --challenge chunin-exams-food-track-cv-2021

In [1]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from cv2 import cv2
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from IPython.display import clear_output

### Reading Dataset 

In [1]:

def resize(image,dim=(150,150)):
    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

def load_dataset(folder, csv_file  ):
    
    mydic = {}
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:rows[1] for rows in reader}
    
    y_train = []
    x_train = []
    count = 1500
    for file in os.listdir(folder):
        if True:
          try:
            # print(file)
            y_train.append((mydict[file]))
            x_train.append(resize(cv2.imread(folder+'/'+file)))
            count -= 1
          except KeyError:
            continue
          
    # print("done") 
    y_train = pd.DataFrame(y_train).astype('category')[0]
    y_train = y_train.cat.codes
    return x_train, y_train.to_numpy()

def load_testdata(folder):
  x_test = []
  for file in os.listdir(folder):
    print(file)
    x_test.append(resize(cv2.imread(folder+'/'+file)))
      

def label_encode(y):
  y = pd.DataFrame(y).astype('category')[0]
  y = y.cat.codes
  return y.to_numpy()  

def get_labels(y_train,y_labels)
  for i in range(len(y_train)):
    y_train[i] = (y_labels[y_train[i]])

## Part 2

In [ ]:
X_train,y_train = load_dataset('/content/drive/MyDrive/Assignment5/train_images','/content/drive/MyDrive/Assignment5/train.csv')
X_test = load_testdata('/content/drive/MyDrive/Assignment5/test_images')

In [23]:
labels = (np.unique(y_train))
y_train = label_encode(y_train)
y_train = to_categorical(y_train)


#### ResNet50

In [ ]:
img = Input(shape = (150, 150)+(3,))
model = ResNet50(
    weights = 'imagenet',include_top = False,
    input_tensor = img, pooling = 'max',classes=61)
output_layer = Dense(61, activation = 'softmax')(model.layers[-1].output)
model = Model(inputs=img, outputs=output_layer)

model.compile(optimizer = 'adam',metrics = ['accuracy'], loss = "sparse_categorical_crossentropy")
model.fit(X_train, y_train, epochs=20, validation_split=0.3) 


Epoch 1/20
204/204 [==============================] - 71s 347ms/step - loss: 2.7403 - accuracy: 0.2884 - val_loss: 3.0704 - val_accuracy: 0.2646
Epoch 2/20
204/204 [==============================] - 71s 349ms/step - loss: 2.6237 - accuracy: 0.3062 - val_loss: 3.7055 - val_accuracy: 0.1194
Epoch 3/20
204/204 [==============================] - 72s 351ms/step - loss: 2.5312 - accuracy: 0.3235 - val_loss: 4.2843 - val_accuracy: 0.0883
Epoch 4/20
204/204 [==============================] - 72s 352ms/step - loss: 2.6729 - accuracy: 0.2979 - val_loss: 3.7927 - val_accuracy: 0.1198
Epoch 5/20
204/204 [==============================] - 72s 352ms/step - loss: 2.4197 - accuracy: 0.3500 - val_loss: 2.8806 - val_accuracy: 0.2703
Epoch 6/20
204/204 [==============================] - 72s 352ms/step - loss: 2.2483 - accuracy: 0.3817 - val_loss: 2.6362 - val_accuracy: 0.3085
Epoch 7/20
204/204 [==============================] - 72s 352ms/step - loss: 2.2358 - accuracy: 0.3950 - val_loss: 3.8049 - val_ac

In [30]:
model.save("/content/drive/MyDrive/Assignment5/model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Assignment5/model/assets


In [28]:
y_pred = model.predict(np.array(X_test))
y_pred = (np.argmax(y_pred, axis = 1))

y_pred = get_labels(y_pred,labels)


In [29]:
df_regressor = pd.DataFrame(y_pred, columns=['ClassName'])
df_regressor.to_csv("/content/drive/MyDrive/Assignment5/submission.csv")

## Making Direct Submission thought Aicrowd CLI

In [ ]:
!aicrowd submission create -c chunin-exams-food-track-cv-2021 -f submission.csv

submission.csv ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 13.1/11.5 KB • 1.4 MB/s • 0:00:00
                                                 ╭─────────────────────────╮                                                  
                                                 │ Successfully submitted! │                                                  
                                                 ╰─────────────────────────╯                                                  
                                                       Important links                                                        
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-7/problems/stage-prediction/submissions/125933              │
│                  │                                                                                                         │
│  All submissions │ https://w